In [1]:
import pandas as pd
import pubchempy as pcp
from tqdm import tqdm
from rdkit import Chem

In [8]:
data = pd.read_csv('Compound_Protein_in_TTD.txt',sep='\t')
Compound_inchikey = list(set(pd.read_csv(r"D:\Herb数据库\version2\edge\Herb_Compound\Herb_Compound_all 0316.csv",index_col=0)['inchikey']))
# cid_set = list(set(data['Pubchem CID']))
# target_set = list(set(data['TTD Target ID']))

In [7]:
# cid_data = pd.DataFrame(cid_set)
# cid_data.columns=['CID']
# cid_data.to_csv('所有cid.csv')

In [3]:
# cid_smiles = pd.read_csv('CID-smiles_TTD.txt',sep='\t',header=None)
# cid_smiles.columns = ['CID','smiles']

In [3]:
cid_smiles = pd.read_csv('CID-smiles_with_canonical_TTD.csv',index_col=0)
cid_smiles['inchikey'] = cid_smiles['Canonical_smiles'].apply(lambda x: Chem.MolToInchiKey(Chem.MolFromSmiles(x)))

In [5]:
# cid_smiles.to_csv('CID-smiles_with_canonical_TTD.csv')


In [4]:
have_cid = list(set(cid_smiles.query("inchikey in @Compound_inchikey")['CID']))

data_have = data.query("`Pubchem CID` in @have_cid")
# data_have.to_csv('有对应cid的靶点-成分对应关系0130.csv')

In [6]:
TTD_info = pd.read_csv('Target_info_in_TTD.txt',sep='\t',header=None)
TTD_have_id = list(set(data_have['TTD Target ID']))
TTD_info.columns = ['id','var','value','drug','phase']
TTD_have_info = TTD_info.query("id in @TTD_have_id and var == 'GENENAME'")[['id','value']]
TTD_have_info.columns = ['TTD Target ID','Gene_name']
data_have_with_tname = pd.merge(data_have,TTD_have_info,how='left',on='TTD Target ID').dropna()
# data_have_with_tname.to_csv('有对应成分的TTD数据（带基因名）0131.csv')
TTD_edge = pd.merge(data_have_with_tname[['Pubchem CID','Gene_name','Activity']],cid_smiles[['CID','Canonical_smiles','inchikey']]
         ,left_on='Pubchem CID',right_on='CID',how='left')[['Gene_name','Canonical_smiles','inchikey','Activity']]

In [9]:
TTD_edge = TTD_edge.loc[TTD_edge[['Gene_name','inchikey']].drop_duplicates().index]
TTD_edge.to_csv('Compound_Protein_filtered_in_TTD.csv')

In [10]:
len(set(TTD_edge['inchikey'])),len(set(TTD_edge['Gene_name']))

(1166, 511)

#### BindingDB

In [11]:
import warnings
warnings.filterwarnings("ignore")
data2 = pd.read_csv('BindingDB_BindingDB_Inhibition.tsv',sep='\t',error_bad_lines=False,warn_bad_lines=False)

In [12]:
data2.columns

Index(['BindingDB Reactant_set_id', 'Ligand SMILES', 'Ligand InChI',
       'Ligand InChI Key', 'BindingDB MonomerID', 'BindingDB Ligand Name',
       'Target Name Assigned by Curator or DataSource',
       'Target Source Organism According to Curator or DataSource', 'Ki (nM)',
       'IC50 (nM)', 'Kd (nM)', 'EC50 (nM)', 'kon (M-1-s-1)', 'koff (s-1)',
       'pH', 'Temp (C)', 'Curation/DataSource', 'Article DOI', 'PMID',
       'PubChem AID', 'Patent Number', 'Authors', 'Institution',
       'Link to Ligand in BindingDB', 'Link to Target in BindingDB',
       'Link to Ligand-Target Pair in BindingDB', 'Ligand HET ID in PDB',
       'PDB ID(s) for Ligand-Target Complex', 'PubChem CID', 'PubChem SID',
       'ChEBI ID of Ligand', 'ChEMBL ID of Ligand', 'DrugBank ID of Ligand',
       'IUPHAR_GRAC ID of Ligand', 'KEGG ID of Ligand', 'ZINC ID of Ligand',
       'Number of Protein Chains in Target (>1 implies a multichain complex)',
       'BindingDB Target Chain  Sequence', 'PDB ID(s) of T

In [13]:
matched_target = list(pd.read_csv('BindingDB靶点匹配结果.tsv',sep='\t')['queryItem'])
matched_data2 = data2.query("`UniProt (SwissProt) Recommended Name of Target Chain` in @matched_target")[['Ligand SMILES',
                    'UniProt (SwissProt) Recommended Name of Target Chain','Ki (nM)','IC50 (nM)', 'Kd (nM)', 'EC50 (nM)',
                    ]]

In [14]:
can_smi2 = []
inchikey2 = []
for smi in tqdm(matched_data2['Ligand SMILES']):
    try:
        can_smi2.append(Chem.MolToSmiles(Chem.MolFromSmiles(smi),kekuleSmiles=True,isomericSmiles=False))
        inchikey2.append(Chem.MolToInchiKey(Chem.MolFromSmiles(smi)))
    except:
        can_smi2.append(None)
        inchikey2.append(None)
        print(smi)

 40%|████      | 23320/57996 [00:20<00:29, 1163.42it/s]

CSc1ccc(cc1)C1=C(C=C[N]([O-])=C1)[C@@H]1CCC(F)(F)C[C@H]1C(=O)NCC#N
CSc1ccc(cc1)C1=C(C=C[N]([O-])=C1)[C@@H]1CCC(F)(F)C[C@H]1C(=O)NCC#N


 74%|███████▎  | 42749/57996 [00:38<00:14, 1037.77it/s]

O=C1NC(=O)c2c1c1c3ccccc3n3[Ru](C#[O])[n+]4cccc2c4c13
CN1C(=O)c2c(C1=O)c1cc(F)c[n+]3[Ru](C#[O])n4c5ccc(O)cc5c2c4c13
CN1C(=O)c2c(C1=O)c1cc(F)c[n+]3[Ru](C#[O])n4c5ccc(O)cc5c2c4c13
O=C1NC(=O)c2c1c1c3ccccc3n3[Ru](C#[O])[n+]4cccc2c4c13
CN1C(=O)c2c(C1=O)c1cc(F)c[n+]3[Ru](C#[O])n4c5ccc(O)cc5c2c4c13
CN1C(=O)c2c(C1=O)c1cc(F)c[n+]3[Ru](C#[O])n4c5ccc(O)cc5c2c4c13


 84%|████████▍ | 48766/57996 [00:43<00:08, 1097.62it/s]

...

 90%|█████████ | 52334/57996 [00:47<00:04, 1157.55it/s]

O=C(N1N=C(CC1c1ccccc1)c1cnc2ccccc12)c1ccccc1
O=C(N1N=C(CC1c1ccccc1)c1cnc2ccccc12)c1ccccc1
O=C(N1N=C(CC1c1ccccc1)c1cnc2ccccc12)c1ccccc1


 98%|█████████▊| 57023/57996 [00:51<00:00, 1313.58it/s]

NS(=O)(=O)c1nnc(s1)N1C(=O)C2=[N](N(C(=C2C(=O)c2ccccc2)c2ccccc2)c2cccc(c2)[N+]([O-])=O)[Co]11N(C(=O)C2=[N]1N(C(=C2C(=O)c1ccccc1)c1ccccc1)c1cccc(c1)[N+]([O-])=O)c1nnc(s1)S(N)(=O)=O
NS(=O)(=O)c1nnc(s1)N1C(=O)C2=[N](N(C(=C2C(=O)c2ccccc2)c2ccccc2)c2cccc(c2)[N+]([O-])=O)[Ni]11N(C(=O)C2=[N]1N(C(=C2C(=O)c1ccccc1)c1ccccc1)c1cccc(c1)[N+]([O-])=O)c1nnc(s1)S(N)(=O)=O
NS(=O)(=O)c1nnc(s1)N1C(=O)C2=[N](N(C(=C2C(=O)c2ccccc2)c2ccccc2)c2cccc(c2)[N+]([O-])=O)[Cu]11N(C(=O)C2=[N]1N(C(=C2C(=O)c1ccccc1)c1ccccc1)c1cccc(c1)[N+]([O-])=O)c1nnc(s1)S(N)(=O)=O
NS(=O)(=O)c1nnc(s1)N1C(=O)C2=[N](N(C(=C2C(=O)c2ccccc2)c2ccccc2)c2cccc(c2)[N+]([O-])=O)[Co]11N(C(=O)C2=[N]1N(C(=C2C(=O)c1ccccc1)c1ccccc1)c1cccc(c1)[N+]([O-])=O)c1nnc(s1)S(N)(=O)=O
NS(=O)(=O)c1nnc(s1)N1C(=O)C2=[N](N(C(=C2C(=O)c2ccccc2)c2ccccc2)c2cccc(c2)[N+]([O-])=O)[Ni]11N(C(=O)C2=[N]1N(C(=C2C(=O)c1ccccc1)c1ccccc1)c1cccc(c1)[N+]([O-])=O)c1nnc(s1)S(N)(=O)=O
NS(=O)(=O)c1nnc(s1)N1C(=O)C2=[N](N(C(=C2C(=O)c2ccccc2)c2ccccc2)c2cccc(c2)[N+]([O-])=O)[Cu]11N(C(=O)C2=[N]

100%|██████████| 57996/57996 [00:52<00:00, 1114.64it/s]


In [15]:
matched_data2['Canonical_smiles'] = can_smi2
matched_data2['inchikey'] = inchikey2

In [21]:
matched_data2.rename(columns={'UniProt (SwissProt) Recommended Name of Target Chain':'queryItem'},inplace=True)
BindingDB_info = pd.read_csv('BindingDB靶点匹配结果.tsv',sep='\t')[['queryItem','preferredName']]
matched_data2_with_tname = pd.merge(matched_data2,BindingDB_info,on='queryItem',how='left')
matched_data2_with_tname.rename(columns={'preferredName':'Gene_name'},inplace=True)
# matched_data2_with_tname.to_csv('BindingDB匹配到靶点的数据.csv')
BindingDB_edge = matched_data2_with_tname.query("inchikey in @Compound_inchikey")[['Canonical_smiles','inchikey','Gene_name','Ki (nM)',
                                                                                                   'IC50 (nM)','Kd (nM)','EC50 (nM)']]

In [24]:
BindingDB_edge = BindingDB_edge.loc[BindingDB_edge[['inchikey','Gene_name']].drop_duplicates().index]
BindingDB_edge.to_csv("Compound_Protein_filtered_in_BindingDB.csv")

In [67]:
len(set(BindingDB_edge['inchikey'])),len(set(BindingDB_edge['Gene_name']))     

(136, 106)

#### STITCH

In [2]:
data3 = pd.read_csv("Compound_Protein_in_STITCH.tsv",sep='\t')
Compound_info_with_target_in_STITCH = pd.read_csv(r"D:\Herb数据库\version2\node\Compound_info_with_target_in_STITCH.csv")
# Compound_info_in_STITCH = pd.read_csv(r"D:\Herb数据库\version2\node\Compound_info_in_STITCH.tsv",sep='\t')

In [3]:
have_link_compound_in_STITCH = list(set(data3['chemical']))
# Compound_info_with_target_in_STITCH = Compound_info_in_STITCH.query("chemical in @have_link_compound_in_STITCH")
# Compound_info_with_target_in_STITCH.to_csv(r"D:\Herb数据库\version2\node\Compound_info_with_target_in_STITCH.csv")

In [6]:
can_smi = []
inchikey = []
for smi in tqdm(Compound_info_with_target_in_STITCH['SMILES_string']):
    try:
        can_smi.append(Chem.MolToSmiles(Chem.MolFromSmiles(smi),kekuleSmiles=True,isomericSmiles=False))
        inchikey.append(Chem.MolToInchiKey(Chem.MolFromSmiles(smi)))
    except:
        can_smi.append(None)
        inchikey.append(None)
Compound_info_with_target_in_STITCH['Canonical_smiles'] = can_smi

100%|██████████| 786494/786494 [11:18<00:00, 1160.00it/s]


In [7]:
Compound_info_with_target_in_STITCH['inchikey'] = inchikey

In [9]:
Compound_info_with_target_and_herb_in_STITCH = Compound_info_with_target_in_STITCH.query("inchikey in @Compound_inchikey")
chemicals_useful = list(set(Compound_info_with_target_and_herb_in_STITCH['chemical']))
data3_with_compound = data3.query("chemical in @chemicals_useful")

In [10]:
target_info_in_STITCH = pd.read_csv(r"D:\Herb数据库\version2\node\Protein_info_in_STRING.txt",sep='\t')[["#string_protein_id",'preferred_name']]
target_info_in_STITCH.rename(columns={"#string_protein_id":'protein','preferred_name':'Gene_name'},inplace=True)

In [11]:
STITCH_edge = pd.merge(
    pd.merge(data3_with_compound[['chemical','protein','combined_score']],
         Compound_info_with_target_and_herb_in_STITCH[['chemical','Canonical_smiles','inchikey']],on='chemical',how='left'),
    target_info_in_STITCH,on='protein',how='left')[['protein','Canonical_smiles','inchikey','Gene_name','combined_score']]

In [12]:
pd.DataFrame(list(set(STITCH_edge.loc[STITCH_edge.Gene_name.isna(),'protein']))).to_csv('Unmatched_protein_in_STITCH.csv')

In [13]:
unmatched_target_info_in_STITCH = pd.read_csv('Unmatched_protein_info_in_STITCH.tsv',sep='\t')[['queryItem','stringId','preferredName']]
unmatched_target_info_in_STITCH.rename(columns={'preferredName':'Gene_name','queryItem':'protein','stringId':'new_protein'},inplace=True)
unmatched_target_info_in_STITCH['protein'] = '9606.'+unmatched_target_info_in_STITCH['protein']

In [15]:
Unmatched_STITCH_edge = pd.merge(STITCH_edge.loc[STITCH_edge.Gene_name.isna(),['protein','Canonical_smiles','inchikey','combined_score']],
                                 unmatched_target_info_in_STITCH,on='protein',how='left')
STITCH_edge_new = pd.concat([STITCH_edge.loc[~STITCH_edge.Gene_name.isna()],Unmatched_STITCH_edge],axis=0)

In [16]:
STITCH_edge_new.loc[~STITCH_edge_new.new_protein.isna(),'protein'] = STITCH_edge_new.loc[~STITCH_edge_new.new_protein.isna(),'new_protein']

In [17]:
STITCH_edge_new.drop('new_protein',axis=1,inplace=True)

In [18]:
STITCH_edge_new = STITCH_edge_new.loc[STITCH_edge_new.dropna()[['inchikey','Gene_name']].drop_duplicates().index]

In [19]:
len(set(STITCH_edge_new.Gene_name)),len(set(STITCH_edge_new.Canonical_smiles))

(16361, 3318)

In [70]:
len(STITCH_edge_new)

390149

In [20]:
STITCH_edge_new.to_csv("Compound_Protein_filtered_in_STITCH.csv")

#### DrugBank

In [21]:
data4 = pd.read_csv('Compound_protein_in_DrugBank.csv').query("Type == 'SmallMoleculeDrug'")
Compound_info_in_DrugBank = pd.read_csv('Compound_info_in_DrugBank.csv')
Target_info_in_DrugBank = pd.read_csv('Target_info_in_DrugBank.csv')

In [22]:
Compound_protein_in_DrugBank = pd.merge(
    pd.merge(data4,Compound_info_in_DrugBank[['DrugBank ID','SMILES']],on='DrugBank ID',how='left'),
    Target_info_in_DrugBank[['UniProt ID','Gene Name']],on='UniProt ID',how='left'
    ).dropna()
can_smi = []
inchikey = []
for smi in Compound_protein_in_DrugBank['SMILES']:
    try:
        can_smi.append(Chem.MolToSmiles(Chem.MolFromSmiles(smi),kekuleSmiles=True,isomericSmiles=False))
        inchikey.append(Chem.MolToInchiKey(Chem.MolFromSmiles(smi)))
    except:
        can_smi.append(None)
        inchikey.append(None)
        print(smi)
Compound_protein_in_DrugBank['Canonical_smiles'] = can_smi
Compound_protein_in_DrugBank['inchikey'] = inchikey

[H][N]([H])([H])[Pt](Cl)(Cl)[N]([H])([H])[H]
[H][N]([H])([H])[Pt](Cl)(Cl)[N]([H])([H])[H]
[H][N]([H])([H])[Pt](Cl)(Cl)[N]([H])([H])[H]
[H][N]([H])([H])[Pt](Cl)(Cl)[N]([H])([H])[H]
NC1=C(C2=C(N)N=C(N)N=C2C=C1)[Cl](=O)=O
NC1=C(C2=C(N)N=C(N)N=C2C=C1)[Cl](=O)=O
C1=CN(C=N1)[Os++]123[N]4=CC=CC=C4C4=[N]1C=CC=C4.C1=CC(C4=CC=CC=[N]24)=[N]3C=C1
[O]#C[Re+]1(C#[O])(C#[O])[N]2=CC=CC3=C2C2=C(C=CC=[N]12)C=C3
[OH2+][Cu-4]([OH2+])([N]1=CNC=C1)([N+]1=CNC=C1)([N+]1=CNC=C1)[N+]1=CNC=C1
F[Al](F)(F)[F-]
F[Al](F)(F)[F-]
F[Al](F)(F)[F-]
F[Al](F)(F)[F-]
F[Al](F)(F)[F-]
F[Al](F)(F)[F-]
F[Al](F)(F)[F-]
F[Al](F)(F)[F-]
F[Al](F)(F)[F-]
F[Al](F)(F)[F-]
CN(CCO[P@](O)(=O)O[P@@](O)(=O)O[Be-](F)(F)F)C1=CC=CC=C1[N+]([O-])=O


In [31]:
DrugBank_edge = Compound_protein_in_DrugBank.query("inchikey in @Compound_inchikey")[['Gene Name','Canonical_smiles','inchikey']].dropna()
DrugBank_edge = DrugBank_edge.loc[DrugBank_edge[['Gene Name','inchikey']].drop_duplicates().index]
DrugBank_edge.rename(columns={'Gene Name':'Gene_name'},inplace=True)

In [25]:
DrugBank_edge.to_csv("Compound_Protein_filtered_in_DrugBank.csv")

In [68]:
len(set(DrugBank_edge['Gene_name'])),len(set(DrugBank_edge.inchikey))

(734, 125)

#### HERB

In [32]:
data5 = pd.read_csv('Compound_Protein_in_HERB 0202.csv',index_col=0)
Target_info_in_HERB = pd.read_csv(r"D:\Herb数据库\version2\node\HERB_target_info.txt",sep='\t')
Compound_info_in_HERB = pd.read_csv(r"D:\Herb数据库\version2\node\HERB_ingredient_info.txt",sep='\t')

In [33]:
STITCH_gene_name = list(target_info_in_STITCH['Gene_name'])
Target_info_in_HERB.query("Gene_name not in @STITCH_gene_name").to_csv("Unmatched_Target_in_HERB.csv")

In [40]:
unmatched_target_info_in_HERB = pd.read_csv("Unmatched_protein_info_in_HERB.tsv",sep='\t',index_col=0)
unmatched_target_info_in_HERB.rename(columns={'queryItem':'Gene_id','preferredName':'Gene_name'},inplace=True)

In [41]:
Target_info_in_HERB_new = pd.concat([Target_info_in_HERB.query("Gene_name in @STITCH_gene_name")[['Target_id','Gene_id','Gene_name']],
           pd.merge(Target_info_in_HERB.query("Gene_name not in @STITCH_gene_name")[['Target_id','Gene_id']],
         unmatched_target_info_in_HERB[['Gene_id','Gene_name']],on='Gene_id',how='inner')],axis=0)

In [42]:
data5_with_info = pd.merge(
    pd.merge(data5,Compound_info_in_HERB[['Ingredient_id','Ingredient_Smile']],on='Ingredient_id',how='left'),
    Target_info_in_HERB_new[['Target_id','Gene_name']],on='Target_id',how='left'
)
can_smi = []
inchikey = []
for smi in tqdm(data5_with_info['Ingredient_Smile']):
    try:
        can_smi.append(Chem.MolToSmiles(Chem.MolFromSmiles(smi),kekuleSmiles=True,isomericSmiles=False))
        inchikey.append(Chem.MolToInchiKey(Chem.MolFromSmiles(smi)))
    except:
        can_smi.append(None)
        inchikey.append(None)
data5_with_info['Canonical_smiles'] = can_smi
data5_with_info['inchikey'] = inchikey


100%|██████████| 183847/183847 [01:42<00:00, 1793.00it/s]


In [43]:
HERB_edge = data5_with_info.query("inchikey in @Compound_inchikey")[['Gene_name','Canonical_smiles','inchikey']].dropna()
HERB_edge = HERB_edge.loc[HERB_edge[['Gene_name','inchikey']].drop_duplicates().index]

In [44]:
HERB_edge.to_csv("Compound_Protein_filtered_in_HERB.csv")

In [45]:
len(set((HERB_edge.Gene_name))),len(set(HERB_edge.Canonical_smiles))

(10559, 2536)

In [47]:
BindingDB_edge = pd.read_csv("Compound_Protein_filtered_in_BindingDB.csv",index_col=0)
TTD_edge = pd.read_csv("Compound_Protein_filtered_in_TTD.csv",index_col=0)

In [54]:
edge_all = pd.concat([BindingDB_edge,TTD_edge,STITCH_edge_new,DrugBank_edge,HERB_edge],axis=0,ignore_index=True).iloc[:,:3].dropna()
edge_all = edge_all.loc[edge_all.iloc[:,1:3].drop_duplicates().index]

In [58]:
len(set((edge_all.Gene_name))),len(set(edge_all.inchikey))

(16626, 4188)

In [59]:
Target_in_network = list(set(pd.read_csv(r"D:\Herb数据库\version2\node\Target_info_in_physical_network_and_experimental.csv",index_col=0)['Gene_name']))
edge_in_network = edge_all.query("Gene_name in @Target_in_network")

In [60]:
edge_in_network.to_csv("Compound_Protein_all 0316.csv")

In [69]:
len(edge_in_network)

297592

In [62]:
len(set(edge_in_network['Gene_name'])),len(set(edge_in_network['inchikey']))

(15903, 4181)

In [63]:
filtered_compound_inchikey = list(set(edge_in_network['inchikey']))
Herb_Compound_data = pd.read_csv(r"D:\Herb数据库\version2\edge\Herb_Compound\Herb_Compound_all 0316.csv",index_col=0)

In [64]:
filtered_Herb_Compound_data = Herb_Compound_data.query("inchikey in @filtered_compound_inchikey")
len(set(filtered_Herb_Compound_data['Herb'])),len(set(filtered_Herb_Compound_data['inchikey']))

(293, 4181)

In [65]:
filtered_Herb_Compound_data.to_csv(r"D:\Herb数据库\version2\edge\Herb_Compound\Herb_Compound_filter 0316.csv")

In [66]:
filtered_Herb_Compound_data.isna()

,Herb,Canonical_smiles,inchikey
0,False,False,False
1,False,False,False
3,False,False,False
4,False,False,False
5,False,False,False
...,...,...,...
27121,False,False,False
27123,False,False,False
27127,False,False,False
27128,False,False,False
